In [2]:
import tensorflow as tf
from python_speech_features import mfcc
import wavio
import numpy as np
import os
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [16]:
#input preprocessing
num_classes = 2; input_size = 13
train_x = []; test_x = []
train_t = []; test_t = []
s0 = wavio.read('at5/d0.wav').data; s0 = mfcc(s0[:,0],44100,0.2,0.1)[:400]
s1 = wavio.read('at5/d1.wav').data; s1 = mfcc(s1[:,0],44100,0.2,0.1)[:400]
d0 = wavio.read('two_people/d0.wav').data; d0 = mfcc(d0[:,0],44100,0.2,0.1)[:400]
d1 = wavio.read('tao,tarun/d0.wav').data; d1 = mfcc(d1[:,0],44100,0.2,0.1)[:400]
train_x = np.concatenate((s0[::2],s1[::2],d0[::2],d1[::2]),0); train_t = [0]*400 + [1]*400
mu = np.mean(train_x,0); sigma = np.std(train_x,0);
train_x = (train_x - mu)/sigma
test_x = np.concatenate((s0[1::2],s1[1::2],d0[1::2],d1[1::2]),0); test_t = [0]*400 + [1]*400
test_x = (test_x - mu)/sigma
n_train = 400
n_test = 400

In [27]:
#hyperparameters
hidden_shape = [200,150]
act_fns = ['tanh','tanh']
net_shape = [input_size] + hidden_shape + [num_classes]
num_lyr = len(hidden_shape)

In [28]:
learning_rate = tf.placeholder(tf.float32,[])
x = tf.placeholder(tf.float32, [None, input_size])
labels = tf.placeholder(tf.int32, [None])
keep_prob = tf.placeholder(tf.float32,[num_lyr+1])

In [29]:
#parameters
params = {'W':{},'b':{}}
for lay_num in range(num_lyr+1):
    params['W'][lay_num] = tf.Variable(tf.random_normal([net_shape[lay_num], net_shape[lay_num+1]],stddev = 0.01))
    params['b'][lay_num] = tf.Variable(tf.random_normal([net_shape[lay_num+1]],stddev = 0.01))

In [30]:
#network graph
def act_fn(a,lay_num):
    if act_fns[lay_num] == 'tanh':
        return tf.tanh(a)
    elif act_fns[lay_num] == 'ReLu':
        return tf.nn.relu(a)
    elif act_fns[lay_num] == 'ELU':
        return tf.nn.elu(a)
    
def MLP(x,params):
    z = x
    for lay_num in range(num_lyr):
        z = act_fn(tf.add(tf.matmul(z,params['W'][lay_num]),params['b'][lay_num]),lay_num)
        z = tf.nn.dropout(z,keep_prob[lay_num])
    logits = tf.add(tf.matmul(z,params['W'][lay_num+1]),params['b'][lay_num+1])
    return tf.nn.dropout(logits,keep_prob[lay_num+1])

In [31]:
logits = MLP(x,params)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits,labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [32]:
sess.run(tf.initialize_all_variables())

In [33]:
n_iter = 0
while True:
    _,cost_ = sess.run([optimizer,cost],feed_dict={x:train_x,labels:train_t,learning_rate:0.1,keep_prob:[0.5,0.5,0.5]})
    if n_iter % 1000 == 0:
        print('iter'+str(n_iter)+' cost:'+str(cost_))
        train_logits = sess.run(logits,feed_dict={x:train_x,keep_prob:[1.0]*3})
        print 'train pf = ' + str((sum(np.argmax(train_logits,axis=1) == train_t) + 0.0)/(n_train*num_classes))
        test_logits = sess.run(logits,feed_dict={x:test_x,keep_prob:[1.0]*3})
        print 'test pf = ' + str((sum(np.argmax(test_logits,axis=1) == test_t) + 0.0)/(n_test*num_classes))+'\n'
    n_iter += 1

iter0 cost:0.69314
train pf = 0.5075
test pf = 0.5025

iter1000 cost:0.570105
train pf = 0.745
test pf = 0.72625

iter2000 cost:0.560639
train pf = 0.7475
test pf = 0.72125

iter3000 cost:0.561453
train pf = 0.745
test pf = 0.72875

iter4000 cost:0.553456
train pf = 0.74625
test pf = 0.72375

iter5000 cost:0.559139
train pf = 0.74625
test pf = 0.72625

iter6000 cost:0.560388
train pf = 0.75
test pf = 0.735

iter7000 cost:0.565641
train pf = 0.75125
test pf = 0.73125

iter8000 cost:0.551961
train pf = 0.7525
test pf = 0.7325

iter9000 cost:0.547078
train pf = 0.75625
test pf = 0.72875

iter10000 cost:0.535064
train pf = 0.755
test pf = 0.72875

iter11000 cost:0.519262
train pf = 0.765
test pf = 0.73125

iter12000 cost:0.539204
train pf = 0.76625
test pf = 0.73

iter13000 cost:0.515729
train pf = 0.7675
test pf = 0.7325

iter14000 cost:0.547094
train pf = 0.7725
test pf = 0.73125

iter15000 cost:0.526223
train pf = 0.7725
test pf = 0.7325

iter16000 cost:0.517477
train pf = 0.78375
test 

KeyboardInterrupt: 

In [205]:
sess.close()

In [35]:
d1 = wavio.read('two_people/d1.wav').data; d1 = mfcc(d1[:,0],44100,0.2,0.1)[:400]; d1 = (d1 - mu)/sigma
d1_logits = sess.run(logits,feed_dict={x:d1,keep_prob:[1.0]*3})
test2 = (mfcc(wavio.read('at5/d0.wav').data[:,0],44100,0.2,0.1)[:200] - mu)/sigma; test2_logits = sess.run(logits,feed_dict={x:test2,keep_prob:[1.0]*3})

In [36]:
np.argmax(d1_logits,axis=1)

array([0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1,

In [37]:
np.argmax(test2_logits,axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0])

In [39]:
j = wavio.read('Jasper/d0.wav').data;j = mfcc(j[:,0],44100,0.2,0.1); j = (j - mu)/sigma
j_logits = sess.run(logits,feed_dict={x:j,keep_prob:[1.0]*3})
j2= (mfcc(wavio.read('Jasper/d1.wav').data[:,0],44100,0.2,0.1) - mu)/sigma; j2_logits = sess.run(logits,feed_dict={x:j2,keep_prob:[1.0]*3})

In [46]:
np.argmax(j_logits,axis=1).shape

(626,)

In [44]:
np.argmax(j2_logits,axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0,

In [48]:
j3= (mfcc(wavio.read('Jasper/d2.wav').data[:,0],44100,0.2,0.1) - mu)/sigma; j3_logits = sess.run(logits,feed_dict={x:j3,keep_prob:[1.0]*3})

In [51]:
np.argmax(j3_logits,axis=1)

(596,)

In [52]:
tt= (mfcc(wavio.read('tao,tarun/d0.wav').data[:,0],44100,0.2,0.1) - mu)/sigma; tt_logits = sess.run(logits,feed_dict={x:tt,keep_prob:[1.0]*3})

In [55]:
np.argmax(tt_logits,axis=1).shape

(677,)